In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('c14').getOrCreate()

In [4]:
my_collection = "Spark The Definitive Guide : Big Data Processing Made Simple".split(" ")

In [5]:
words = spark.sparkContext.parallelize(my_collection, 2)

In [6]:
supplementalData = {
    "Spark" : 1000,
    "Definitive" : 200,
    "Big" : -300,
    "Simple" : 100
}

In [7]:
# broadcast

In [8]:
suppBroadcast = spark.sparkContext.broadcast(supplementalData)

In [9]:
suppBroadcast.value

{'Big': -300, 'Definitive': 200, 'Simple': 100, 'Spark': 1000}

In [10]:
# using in rdd

In [11]:
words.map(lambda word: (word, suppBroadcast.value.get(word,0)))\
.sortBy(lambda wordPair: wordPair[1])\
.collect()

[('Big', -300),
 ('The', 0),
 ('Guide', 0),
 (':', 0),
 ('Data', 0),
 ('Processing', 0),
 ('Made', 0),
 ('Simple', 100),
 ('Definitive', 200),
 ('Spark', 1000)]

In [12]:
# accumulators

In [13]:
flights = spark.read.parquet('flight-data/parquet/2010-summary.parquet')

In [14]:
#unnamed accumulators

In [15]:
accChina = spark.sparkContext.accumulator(0)

In [16]:
def accChinaFunc(flight_row):
    destination = flight_row["DEST_COUNTRY_NAME"]
    origin = flight_row["ORIGIN_COUNTRY_NAME"]
    if destination == "China":
        accChina.add(flight_row['count'])
    if origin == "China":
        accChina.add(flight_row['count'])

In [17]:
flights.foreach(lambda flight_row: accChinaFunc(flight_row))

In [18]:
accChina.value

953